# Tutorial: Busca Semântica Inteligente com LangChain e Groq

Este tutorial mostra como criar um buscador inteligente de informações em documentos usando LangChain e Groq. O objetivo é encontrar respostas relevantes em grandes volumes de texto, resolvendo o problema de buscas tradicionais que não entendem o significado das palavras.

Você vai aprender:
- Por que usar busca semântica
- Como configurar o ambiente
- Como carregar e dividir documentos
- Como gerar embeddings com Groq
- Como buscar respostas relevantes

## Por que busca semântica?

Buscas tradicionais (por palavra-chave) não entendem contexto ou significado. Busca semântica usa IA para entender o sentido das perguntas e encontrar respostas mesmo que as palavras não sejam idênticas.

**Impacto:** Facilita encontrar informações relevantes em textos longos, como relatórios, artigos ou PDFs.

## 1. Instalação dos Pacotes

Vamos instalar as bibliotecas necessárias para usar LangChain, Groq e manipular PDFs.

In [1]:
# Instale as bibliotecas necessárias
!pip install langchain groq langchain-community pypdf


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Configuração da API Groq

Para usar modelos Groq, você precisa de uma chave de API. Isso garante acesso seguro e rastreável aos recursos de IA.

In [2]:
import os
import getpass

# Defina sua chave de API Groq (ela pode ser obtida no site da Groq)
os.environ['GROQ_API_KEY'] = getpass.getpass('Digite sua chave de API Groq:')

## 3. Carregando Documentos

Vamos carregar um PDF para buscar informações. O loader transforma cada página em um documento separado.

**Problema resolvido:** Automatiza a leitura de grandes arquivos, tornando-os pesquisáveis.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = 'Docs/nke-10k-2023.pdf'  # Caminho para seu PDF
loader = PyPDFLoader(file_path)
docs = loader.load()
print(f'Total de páginas carregadas: {len(docs)}')

Total de páginas carregadas: 107


## 4. Dividindo o Texto em Chunks

Dividimos o texto em pedaços menores para facilitar a busca e garantir que o modelo não ultrapasse limites de tamanho.

**Finalidade:** Permite buscas mais precisas e rápidas.

**Exemplo:** Um relatório de 10 páginas pode virar 30 pedaços de texto.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f'Total de chunks: {len(chunks)}')

Total de chunks: 516


## 5. Gerando Embeddings dos Textos

Embeddings são vetores que representam o significado dos textos. Usamos modelos como HuggingFace para gerar esses vetores, pois o Groq é voltado para geração de texto, não para embeddings.

**Impacto:** Permite comparar perguntas e textos pelo significado, não só por palavras exatas.

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

# Cria o objeto de embeddings usando um modelo HuggingFace (especializado em transformar texto em vetores)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

c:\Users\Guilh\OneDrive\Área de Trabalho\Projetos IA\TutorialChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 6. Criando o Banco Vetorial

O banco vetorial armazena os embeddings dos chunks. Assim, é possível buscar rapidamente os trechos mais relevantes para uma pergunta.

**Problema resolvido:** Busca eficiente em grandes volumes de texto.

In [7]:
# Antes de usar FAISS, é necessário instalar o pacote faiss-cpu (ou faiss-gpu se tiver GPU compatível).
# Execute a linha abaixo se ainda não instalou:
!pip install faiss-cpu

from langchain.vectorstores import FAISS

# Cria o banco vetorial com os embeddings dos chunks
vector_store = FAISS.from_documents(chunks, embeddings)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

## 7. Fazendo uma Busca Semântica

Agora podemos buscar por significado, não só por palavras exatas.

**Exemplo:** Buscar 'quantos centros de distribuição a Nike tem nos EUA?'

In [ ]:
query = 'How many distribution centers does Nike have in the US?'
results = vector_store.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f'--- Resultado {i} ---')
    print(doc.page_content[:300])
    print(doc.metadata)
    print()

## Conclusão

Com Groq e LangChain, você pode criar buscadores inteligentes que entendem o significado das perguntas, tornando a busca em documentos muito mais eficiente e útil.

**Problema resolvido:** Encontrar rapidamente informações relevantes em grandes volumes de texto, sem depender de palavras exatas.